In [ ]:
from datetime import datetime, timedelta
from dateutil.parser import parse as parseDate
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

## Open the csv file

In [ ]:
columnInfo = {
    "Id": "Int64",
    "D": str,
    "T": "Int64",
    "NB": "Int64",
    "V": "Int64",
    "PI": "Int64",
    "Nboc": "Int64",
    "Iddp": "Int64"
}

def divideByTen(x):
    return int(x) / 10

data = pd.read_csv('/kaggle/input/donnees-plc-rablire-saintalfred-2021/Erabliere_Saint-Alfred_Saison_2021.csv', 
                   sep = ';',
                   header = 0,
                   names = ["Id", "DateTime", "Temperature", "NiveauBassin", "Vaccium", "PlusGrandInterval", "NombreOccurence", "IdDonneePlusRecente"],
                   usecols = ["DateTime", "Temperature", "NiveauBassin", "Vaccium"],
                   converters = {"DateTime": parseDate, "Temperature": divideByTen, "Vaccium": divideByTen},
                   dtype=columnInfo) 
data

In [ ]:
data.info()

In [ ]:
data.describe()

## How many hours the temperature was greater than 0 degre celcius ?

In [ ]:
lastDate = None
temperatureGreaterThatZero = timedelta()
temperatureLesserThatZero = timedelta()

for index, row in data.iterrows():
    if lastDate == None:
        lastDate = row['DateTime']
    elif row['Temperature'] >= 0:
        temperatureGreaterThatZero += row['DateTime'] - lastDate
        lastDate = row['DateTime']
    else:
        temperatureLesserThatZero += row['DateTime'] - lastDate
        lastDate = row['DateTime']

In [ ]:
print('Between the ', data['DateTime'][0], 'and the', lastDate)
print('Temperature greater that zero:', temperatureGreaterThatZero)
print('Temperature lesser that zero:', temperatureLesserThatZero)
print()
print('In hours this represent')
print('Temperature greater that zero in hours:', temperatureGreaterThatZero.total_seconds() / 60 / 60)
print('Temperature lesser that zero in hours:', temperatureLesserThatZero.total_seconds() / 60 / 60)

## How many hours the temperature is greater that 0 by day ?

In [ ]:
lastDate = None
days = []

for index, row in data.iterrows():
    if lastDate == None:
        lastDate = row['DateTime']
        days.append(0)
        continue
    
    temperatureGreaterThatZero = 0
    
    if row['Temperature'] >= 0:
        temperatureGreaterThatZero += (row['DateTime'] - lastDate).total_seconds() / 60 / 60
    
    if lastDate.date() != row['DateTime'].date():
        days.append(temperatureGreaterThatZero)
    else:
        days[len(days) - 1] += temperatureGreaterThatZero
    
    lastDate = row['DateTime']

In [ ]:
plt.title("Hours per days where temperature was greater or equal to 0 degre celcius")
plt.plot(days)
plt.ylabel('Hours T >= 0 celcius')
plt.show()